# LUIS

Pour utiliser l'application LUIS, nous devons d'abord créer une ressource (après avoir créé un compte Azure).

![Ressource](image_notebook/LUIS/creer-ressource.png)

----

![Ressource2](image_notebook/LUIS/creer-ressource2.png)

----

![Ressource3](image_notebook/LUIS/creer-ressource3.png)

Nous utilisons l'abonnement que nous avons choisi lors de la création de notre compte. Pour les exemples, le groupe de ressource se nommera "test".

Nous cliquons ensuite sur le bouton "Vérifier + Créer" en bas de l'écran et sur "Créer".

![Ressource4](image_notebook/LUIS/creer-ressource4.png)

Nous avons maintenant notre groupe de ressources "Test", nous pouvons ensuite cliquer dessus.

![Ressource5](image_notebook/LUIS/creer-ressource5.png)

Une fois sur la page de notre groupe de ressources, nous pouvons créer des ressources, nous cliquons sur le bouton.

![Ressource-luis](image_notebook/LUIS/ressource-luis.png)

Nous tapons ensuite dans la barre de recherche "Langue Understanding" puis nous cliquons sur la première proposition.

![Ressource-luis2](image_notebook/LUIS/ressource-luis2.png)

Nous cliquons ensuite sur "Créer".

![Ressource-luis3](image_notebook/LUIS/ressource-luis3.png)

Nous avons ensuite deux possibilités au niveau des options de création.

L'option de création est : 
>Pour créer, gérer, entrainer, tester et publier votre modèle LUIS, vous devez créer la ressource de création ci-dessous. Vous bénéficiez alors de 1 000 demandes de point de terminaison/mois.

L'option de prédiction est : 
>Si vous voulez donner à votre application cliente l'accès à plus de demandes que les 1 000 fournies par la création, créez la ressource de prédiction ci-dessous.

Et l'option pour choisir les deux.

Dans notre cas nous avons utilisé l'option de création et de prédiction, nous aurions dû directement choisir l'option "les deux". Nous devons bien faire attention à utiliser le **niveau tarifaire gratuit lors de la création de notre ressource**.

Puis nous cliquons sur "Vérifier + Créer" et "Créer".

![Ressource-luis4](image_notebook/LUIS/ressource-luis4.png)

## LUIS via le portail luis.ai
Nous avons maintenant notre ressource Luis. Nous avons ensuite été sur le portail LUIS à cette adresse : https://www.luis.ai/ où nous avons créé un compte.

![portail-luis](image_notebook/LUIS/portail-luis.png)

Il nous a suffi ensuite de créer une nouvelle application en entrant le formulaire et en sélectionnant notre ressource de prédiction que nous avons créée sur Azure.

Une fois l'application créer, il suffit de cliquer dessus pour se retrouver sur la page de l'application. D'aller ensuite dans l'onglet "Prebuilt Domain" et de sélectionner "RestaurantReservation".

![portail-luis2](image_notebook/LUIS/portail-luis2.png)

Nous pouvons ensuite entrainer avec le bouton "Train" en haut à droite de l'écran puis sur "Publish" en sélectionnant "Production".

![portail-luis3](image_notebook/LUIS/portail-luis3.png)

Nous avons ici exploré LUIS, ce qui a été fait n'est pas demandé dans le projet, mais nous a permit d'avoir une vue d'ensemble et de comprendre comment cela fonctionnait.

## LUIS via SDK Luis Python

Nous avons pour cette partie recréé une ressource LUIS avec pour option "les deux".

Avant de commencer cette partie, nous devons définir ce que sont ces trois termes : 
- **Utterance**: le message texte brut ex: Book a hotel in Seattle
- **Intent**: qu'est-ce que l'utilisateur essaie de faire ex.: réserver un hôtel
- **Entity**: à quoi l'utilisateur fait-il référence ? Ex.: Seatt

## Import des librairies

In [1]:
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
from functools import reduce

import pandas as pd

import json, time, uuid
import os
from dotenv import load_dotenv

In [3]:
authoringKey = os.environ.get("APP_AUTHORING_KEY")
authoringEndpoint = os.environ.get("ENDPOINT_AUTHORING_URL")

appName = 'Luis-P10-Reda-Authoring'
versionId = "0.1"
intentName = "Booking"

Nous créons un objet **CognitiveServicesCredentials** avec notre clé et l'utilisons avec notre point de terminaison pour créer un objet **LUISAuthoringClient**.

In [4]:
client = LUISAuthoringClient(authoringEndpoint,
                             CognitiveServicesCredentials(authoringKey))

In [5]:
# define app basics
appDefinition = ApplicationCreateObject(name=appName,
                                        initial_version_id=versionId,
                                        culture='en-us')

# create app
app_id = client.apps.add(appDefinition)

# get app id - necessary for all other changes
print("Created LUIS app with ID {}".format(app_id))

Created LUIS app with ID 07c3e708-6145-47c4-8e53-95b894f863ff


Nous avons maintenant notre application de créée.

![luis-sdk-create-app](image_notebook/LUIS/luis-sdk-create-app.png)

Nous créons maintenant l'intention de l'application.

In [6]:
# app_id = "9173b5d9-15dd-4b11-8dca-969126d1d2ad"

client.model.add_intent(app_id, versionId, intentName)

'1b1efe7a-e75e-4e20-bc59-14cf8d5b659d'

Nous avons maintenant notre intention (indent).

![luis-sdk-create-app-indent](image_notebook/LUIS/luis-sdk-create-app-indent.png)

Créons maintenant l'entité de l'application.

In [7]:
def get_grandchild_id(model, childName, grandChildName):

    theseChildren = next(
        filter((lambda child: child.name == childName), model.children))
    theseGrandchildren = next(
        filter((lambda child: child.name == grandChildName),
               theseChildren.children))

    grandChildId = theseGrandchildren.id

    return grandChildId

In [8]:
# Add Prebuilt entity
# client.model.add_prebuilt(app_id, versionId, prebuilt_extractor_names=['datetimeV2', 'money', 'geographyV2'])

# define machine-learned entity
mlEntityDefinition = [{
    "name":
    "Fly",
    "children": [{
        "name": "or_city"
    }, {
        "name": "dst_city"
    }, {
        "name": "str_date"
    }, {
        "name": "end_date"
    }, {
        "name": "budget"
    }]
}]

# add entity to app
modelId = client.model.add_entity(app_id,
                                  versionId,
                                  name="FlyOrder",
                                  children=mlEntityDefinition)

# define phraselist - add phrases as significant vocabulary to app
# Ajoute un poids aux mots

phraseList_dst = {
    "enabledForAllModels": False,
    "isExchangeable": True,
    "name": "Phraselist",
    "phrases": "fly, destination, hotel"
}

# add phrase list to app
phraseList_dst = client.features.add_phrase_list(app_id, versionId,
                                                 phraseList_dst)

# add phrase list as feature to subentity model
modelObject = client.model.get_entity(app_id, versionId, modelId)

FlyDst_cityId = get_grandchild_id(modelObject, "Fly", "dst_city")

phraseListFeatureDefinition = {
    "feature_name": "Phraselist",
    "model_name": None
}
client.features.add_entity_feature(app_id, versionId, FlyDst_cityId,
                                   phraseListFeatureDefinition)

Nous avons maintenant notre entité.

![app-entite](image_notebook/LUIS/app-entite.png)
![app-entite2](image_notebook/LUIS/app-entite2.png)

Nous avons maintenant notre entité ainsi que notre feature attribué à "dst_city".

Nous allons maintenant ajouter un exemple d'énoncé.

Comme dit dans la documentation :
>Pour déterminer l’intention d’un énoncé et extraire des entités, l’application a besoin d’exemples d’énoncés. Les exemples doivent cibler une intention spécifique et unique, et doivent marquer toutes les entités personnalisées. Les entités prédéfinies n’ont pas besoin d’être marquées.

>Ajoutez des exemples d’énoncés en créant une liste d’objets ExampleLabelObject, un objet pour chaque exemple d’énoncé. Chaque exemple doit marquer toutes les entités avec un dictionnaire de paires nom/valeur de nom d’entité et de valeur d’entité. La valeur de l’entité doit être exactement telle qu’elle apparaît dans le texte de l’exemple d’énoncé.

Adaptons l'exemple de la documentation à notre projet. Importons notre dataframe train_df pour en extraire un turn, de manière à avoir un exemple un peu plus concret.

In [9]:
df = pd.read_csv('data/train_df.csv')
df.head()

,text,or_city,dst_city,str_date,end_date,budget
0,I am a world-renowned pastry chef and I need t...,NaN,St. Petersburg,NaN,NaN,NaN
1,IM IN TIJUANA FIND ME A FLIGHT TO CURITIBA AUG...,TIJUANA,CURITIBA,AUG 27,SEPT 4,NaN
2,Hi im from punta cana looking to go on a trip,punta cana,NaN,NaN,NaN,NaN
3,Where can this guy go for 2400??? I live in Me...,Mexico City,Milan,NaN,NaN,2400
4,Are there 5 star hotels in Kyoto?,NaN,Kyoto,NaN,NaN,NaN


Prenons le deuxième texte pour exemple. Ajoutons seulement un budget factice.

In [10]:
text = df["text"][1] + " for a budget of 3500 dollars"
print(len(text))
text

88


'IM IN TIJUANA FIND ME A FLIGHT TO CURITIBA AUG 27 TO SEPT 4 for a budget of 3500 dollars'

Nous devons maintenant créer notre exemple, avant cela, définissons les différentes clefs du format attendu :
- **text**: Le message envoyé par l'utilisateur
- **intentName**: Enoncé de la demande
- **entityLabels**: Les différents labels de l'entités (or_city, dst_city...)
- **startCharIndex**: Début des informations à récupérer par le chatbot
- **endCharIndex**: Fin des informations à récupérer par le chatbot
- **entityName**: Nom de l'entité

In [11]:
# Define labeled example
labeledExampleUtteranceWithMLEntity = {
    "text":
    text,
    "intentName":
    intentName,
    "entityLabels": [{
        "startCharIndex":
        6,
        "endCharIndex":
        87,
        "entityName":
        "FlyOrder",
        "children": [{
            "startCharIndex":
            6,
            "endCharIndex":
            87,
            "entityName":
            "Fly",
            "children": [{
                "startCharIndex": 6,
                "endCharIndex": 13,
                "entityName": "or_city"
            }, {
                "startCharIndex": 34,
                "endCharIndex": 42,
                "entityName": "dst_city"
            }, {
                "startCharIndex": 43,
                "endCharIndex": 49,
                "entityName": "str_date"
            }, {
                "startCharIndex": 53,
                "endCharIndex": 59,
                "entityName": "end_date"
            }, {
                "startCharIndex": 76,
                "endCharIndex": 87,
                "entityName": "budget"
            }]
        }]
    }]
}

print("Labeled Example Utterance:", labeledExampleUtteranceWithMLEntity)

# Add an example for the entity.
# Enable nested children to allow using multiple models with the same name.
# The quantity subentity and the phraselist could have the same exact name if this is set to True
client.examples.add(app_id, versionId, labeledExampleUtteranceWithMLEntity,
                    {"enableNestedChildren": True})

Labeled Example Utterance: {'text': 'IM IN TIJUANA FIND ME A FLIGHT TO CURITIBA AUG 27 TO SEPT 4 for a budget of 3500 dollars', 'intentName': 'Booking', 'entityLabels': [{'startCharIndex': 6, 'endCharIndex': 87, 'entityName': 'FlyOrder', 'children': [{'startCharIndex': 6, 'endCharIndex': 87, 'entityName': 'Fly', 'children': [{'startCharIndex': 6, 'endCharIndex': 13, 'entityName': 'or_city'}, {'startCharIndex': 34, 'endCharIndex': 42, 'entityName': 'dst_city'}, {'startCharIndex': 43, 'endCharIndex': 49, 'entityName': 'str_date'}, {'startCharIndex': 53, 'endCharIndex': 59, 'entityName': 'end_date'}, {'startCharIndex': 76, 'endCharIndex': 87, 'entityName': 'budget'}]}]}]}


Nous avons maintenant notre example.

![Luis_exemple](image_notebook/LUIS/concept_exemple.png)

Passons au premier entrainement de notre modèle.

In [12]:
client.train.train_version(app_id, versionId)
waiting = True
while waiting:
    info = client.train.get_status(app_id, versionId)

    # get_status returns a list of training statuses, one for each model. Loop through them and make sure all are done.
    waiting = any(
        map(
            lambda x: 'Queued' == x.details.status or 'InProgress' == x.details
            .status, info))
    if waiting:
        print("Waiting 10 seconds for training to complete...")
        time.sleep(10)
    else:
        print("trained")
        waiting = False

Waiting 10 seconds for training to complete...
trained


Publions notre modèle.

In [13]:
# Mark the app as public so we can query it using any prediction endpoint.
# Note: For production scenarios, you should instead assign the app to your own LUIS prediction endpoint. See:
# https://docs.microsoft.com/en-gb/azure/cognitive-services/luis/luis-how-to-azure-subscription#assign-a-resource-to-an-app
client.apps.update_settings(app_id, is_public=True)

responseEndpointInfo = client.apps.publish(app_id, versionId, is_staging=False)

Testons une prédiction. **Pour le code final, nous devons penser utiliser la clef API de prédiction pour interroger l'application**.

In [14]:
runtimeCredentials = CognitiveServicesCredentials(authoringKey)
clientRuntime = LUISRuntimeClient(endpoint=authoringEndpoint,
                                  credentials=runtimeCredentials)

In [15]:
text_prediction = "IM IN UKRAINE FIND ME A FLIGHT TO MALAYSIA FEB 22 TO SEPT 8 for a budget of 6500 dollars"

In [16]:
# Production == slot name
predictionRequest = {"query": text_prediction}

predictionResponse = clientRuntime.prediction.get_slot_prediction(
    app_id, "Production", predictionRequest)
print("Top intent: {}".format(predictionResponse.prediction.top_intent))
print("Sentiment: {}".format(predictionResponse.prediction.sentiment))
print("Intents: ")

for intent in predictionResponse.prediction.intents:
    print("\t{}".format(json.dumps(intent)))
print("Entities: {}".format(predictionResponse.prediction.entities))

Top intent: Booking
Sentiment: None
Intents: 
	"Booking"
Entities: {'FlyOrder': [{'Fly': [{'or_city': ['UKRAINE'], 'dst_city': ['MALAYSIA'], 'str_date': ['FEB 22'], 'end_date': ['SEPT 8'], 'budget': ['6500 dollars']}]}]}


Pour ce premier entrainement et ce premier test, nous avons des résultats plutôt convaincants.

Essayons maintenant de créer une autre application avec pour entrainement notre données train et avec pour test nos données de test.